In [1]:
!git clone https://github.com/MohamedKhalifa1/Stack-Overflow-Annual-Developer-Survey-Analysis

Cloning into 'Stack-Overflow-Annual-Developer-Survey-Analysis'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 99 (delta 22), reused 42 (delta 13), pack-reused 47
Receiving objects: 100% (99/99), 31.21 MiB | 33.53 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [2]:
!unzip /content/Stack-Overflow-Annual-Developer-Survey-Analysis/data/preprocessed/01_preprocessed_data.zip

Archive:  /content/Stack-Overflow-Annual-Developer-Survey-Analysis/data/preprocessed/01_preprocessed_data.zip
  inflating: content/01_preprocessed_data.pkl  


In [3]:
DATA_PATH = '/content/content/01_preprocessed_data.pkl'
ROLES_PATH = '/content/Stack-Overflow-Annual-Developer-Survey-Analysis/data/raw/roles_short_names.csv'
IMAGES_PATH = '/content/Stack-Overflow-Annual-Developer-Survey-Analysis/img'

In [4]:
ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageHaveWorkedWith',
                  'DatabaseHaveWorkedWith',
                  'PlatformHaveWorkedWith',
                  'WebframeHaveWorkedWith',
                  'MiscTechHaveWorkedWith',
                  'ToolsTechHaveWorkedWith',
                  'NEWCollabToolsHaveWorkedWith']

In [5]:
# !unzip /content/Stack-Overflow-Annual-Developer-Survey-Analysis/data/raw/stack-overflow-developer-survey-2023.zip -d /content/Stack-Overflow-Annual-Developer-Survey-Analysis/data/raw

In [6]:
%cd /content/Stack-Overflow-Annual-Developer-Survey-Analysis

/content/Stack-Overflow-Annual-Developer-Survey-Analysis


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import MultiLabelBinarizer
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Read data and preprocess

In [8]:
raw_df  = pd.read_pickle(DATA_PATH)
roles_names = pd.read_csv(ROLES_PATH , sep=';')

In [9]:
raw_df.head()

,ResponseId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,ProfessionalTech,Industry,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I agree,None of these,18-24 years old,[],NaN,[],NaN,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,2,I agree,I am a developer by profession,25-34 years old,"[Employed, full-time]",Remote,"[Hobby, Contribute to open-source projects, Bo...","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","[Books / Physical media, Colleague, Friend or ...",[Formal documentation provided by the owner of...,...,1-2 times a week,10+ times a week,Never,15-30 minutes a day,15-30 minutes a day,"[DevOps function, Microservices, Automated tes...","Information Services, IT, Software Development...",Appropriate in length,Easy,285000.0
2,3,I agree,I am a developer by profession,45-54 years old,"[Employed, full-time]","Hybrid (some remote, some in-person)","[Hobby, Professional development or self-paced...","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","[Books / Physical media, Colleague, On the job...",[Formal documentation provided by the owner of...,...,6-10 times a week,6-10 times a week,3-5 times a week,30-60 minutes a day,30-60 minutes a day,"[DevOps function, Microservices, Automated tes...","Information Services, IT, Software Development...",Appropriate in length,Easy,250000.0
3,4,I agree,I am a developer by profession,25-34 years old,"[Employed, full-time]","Hybrid (some remote, some in-person)",[Hobby],"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","[Colleague, Friend or family member, Other onl...",[Formal documentation provided by the owner of...,...,1-2 times a week,10+ times a week,1-2 times a week,15-30 minutes a day,30-60 minutes a day,"[Automated testing, Continuous integration (CI...",NaN,Appropriate in length,Easy,156000.0
4,5,I agree,I am a developer by profession,25-34 years old,"[Employed, full-time, Independent contractor, ...",Remote,"[Hobby, Contribute to open-source projects, Pr...","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","[Books / Physical media, Online Courses or Cer...",[Formal documentation provided by the owner of...,...,1-2 times a week,1-2 times a week,3-5 times a week,60-120 minutes a day,30-60 minutes a day,"[Microservices, Automated testing, Observabili...",Other,Appropriate in length,Neither easy nor difficult,23456.0


In [10]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89184 entries, 0 to 89183
Data columns (total 84 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ResponseId                           89184 non-null  int64  
 1   Q120                                 89184 non-null  object 
 2   MainBranch                           89184 non-null  object 
 3   Age                                  89184 non-null  object 
 4   Employment                           89184 non-null  object 
 5   RemoteWork                           73810 non-null  object 
 6   CodingActivities                     89184 non-null  object 
 7   EdLevel                              87973 non-null  object 
 8   LearnCode                            89184 non-null  object 
 9   LearnCodeOnline                      89184 non-null  object 
 10  LearnCodeCoursesCert                 89184 non-null  object 
 11  YearsCode                   

In [11]:
raw_df.describe()

,ResponseId,YearsCode,YearsCodePro,CompTotal,WorkExp,ConvertedCompYearly
count,89184.000000,87435.000000,66136.000000,4.822500e+04,43579.000000,4.801900e+04
mean,44592.500000,13.977926,11.266194,1.036807e+42,11.405126,1.031101e+05
std,25745.347541,10.538807,9.211307,2.276847e+44,9.051989,6.814188e+05
min,1.000000,1.000000,1.000000,0.000000e+00,0.000000,1.000000e+00
25%,22296.750000,6.000000,4.000000,6.300000e+04,5.000000,4.390700e+04
50%,44592.500000,11.000000,9.000000,1.150000e+05,9.000000,7.496300e+04
75%,66888.250000,20.000000,16.000000,2.300000e+05,16.000000,1.216410e+05
max,89184.000000,51.000000,51.000000,5.000000e+46,50.000000,7.435143e+07


In [12]:
roles_names.head()

,Original name,Short name
0,"Developer, back-end",Back-end dev
1,"Developer, full-stack",Full-stack dev
2,"Developer, front-end",Front-end dev
3,"Developer, desktop or enterprise applications",Desktop dev
4,"Developer, mobile",Mobile dev


In [13]:
print(raw_df['DevType'].unique())

[nan 'Senior Executive (C-Suite, VP, etc.)' 'Developer, back-end'
 'Developer, front-end' 'Developer, full-stack' 'System administrator'
 'Developer, desktop or enterprise applications' 'Developer, QA or test'
 'Designer' 'Data scientist or machine learning specialist'
 'Data or business analyst' 'Security professional' 'Educator'
 'Research & Development role' 'Other (please specify):'
 'Developer, mobile' 'Database administrator'
 'Developer, embedded applications or devices' 'Student' 'Engineer, data'
 'Hardware Engineer' 'Product manager' 'Academic researcher'
 'Developer, game or graphics' 'Cloud infrastructure engineer'
 'Engineering manager' 'Developer Experience' 'Project manager'
 'DevOps specialist' 'Engineer, site reliability' 'Blockchain'
 'Developer Advocate' 'Scientist' 'Marketing or sales professional']


In [14]:
print(roles_names['Short name '].unique())

['Back-end dev' 'Full-stack dev' 'Front-end dev' 'Desktop dev'
 'Mobile dev' 'DevOps' 'Database admin' 'Designer' 'System admin'
 'Embedded dev' 'Analyst' 'Data Scientist ' 'Quality Assurance'
 'Data Engineer' 'Researcher' 'Educator' 'Game dev' 'Engineering manager'
 'Product manager' 'Scientist' 'Site Engineer' 'Senior Executive'
 'Marketing/Sales' 'Student' 'Other']


In [15]:
roles_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Original name  25 non-null     object
 1   Short name     25 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes


In [16]:
df = raw_df.copy()
encoded_dfs = {}
for col in ROLE_COLS + TECH_COLS:
    df[col].fillna('', inplace=True)

    df[col] = df[col].apply(lambda x: x if isinstance(x, list) else [x] if pd.notnull(x) else [])

    binarizer = MultiLabelBinarizer()
    encoded_df = pd.DataFrame(binarizer.fit_transform(df[col]),
                               columns=binarizer.classes_,
                               index=df.index)
    encoded_dfs[col] = encoded_df

In [17]:
df = pd.concat(encoded_dfs, axis=1)

In [18]:
df.head()

DevType                                                               \
          Academic researcher Blockchain Cloud infrastructure engineer   
0       1                   0          0                             0   
1       0                   0          0                             0   
2       0                   0          0                             0   
3       0                   0          0                             0   
4       0                   0          0                             0   

                                                                          \
  Data or business analyst Data scientist or machine learning specialist   
0                        0                                             0   
1                        0                                             0   
2                        0                                             0   
3                        0                                             0   
4                        0                                             0   

                                                                        ...  \
  Database administrator Designer DevOps specialist Developer Advocate  ...   
0                      0        0                 0                  0  ...   
1                      0        0                 0                  0  ...   
2                      0        0                 0                  0  ...   
3                      0        0                 0                  0  ...   
4                      0        0                 0                  0  ...   

  NEWCollabToolsHaveWorkedWith                                     \
                        Spyder Sublime Text TextMate VSCodium Vim   
0                            0            0        0        0   0   
1                            0            0        0        0   1   
2                            0            0        0        0   0   
3                            0            0        0        0   1   
4                            0            0        0        0   1   

                                                         
  Visual Studio Visual Studio Code WebStorm Xcode condo  
0             0                  0        0     0     0  
1             0                  1        0     0     0  
2             0                  0        0     0     0  
3             0                  1        1     0     0  
4             0                  1        0     0     0  

[5 rows x 304 columns]

In [19]:
jobs_freq = df['DevType'].copy().sum().sort_values().reset_index()
jobs_freq

,index,0
0,Marketing or sales professional,149
1,Developer Advocate,212
2,Database administrator,257
3,Designer,281
4,Hardware Engineer,286
5,Blockchain,323
6,Developer Experience,326
7,Scientist,351
8,Educator,415
9,"Engineer, site reliability",427


# Display Skills Frequancy

In [20]:
skills_freq = df.copy().drop('DevType', axis=1).sum().sort_values(ascending=False).reset_index()
skills_freq.columns = ['group', 'skill', 'freq']

In [21]:
skills_freq

,group,skill,freq
0,NEWCollabToolsHaveWorkedWith,Visual Studio Code,63793
1,LanguageHaveWorkedWith,JavaScript,55711
2,LanguageHaveWorkedWith,HTML/CSS,46396
3,LanguageHaveWorkedWith,Python,43158
4,LanguageHaveWorkedWith,SQL,42623
...,...,...,...
265,ToolsTechHaveWorkedWith,snitch,55
266,ToolsTechHaveWorkedWith,lest,55
267,ToolsTechHaveWorkedWith,CUTE,54
268,ToolsTechHaveWorkedWith,ELFspy,51


In [22]:
!mkdir -p /content/Stack-Overflow-Annual-Developer-Survey-Analysis/img

In [23]:
fig = px.treemap(skills_freq,
                 title='Skills Frequancy',
                 path=['group', 'skill'],
                 color_continuous_scale='deep',
                 values='freq',color='freq')

fig.update_layout(width=1700, height=800)
fig.show()
fig.write_html(f'{IMAGES_PATH}/skills_freq.html')

In [24]:
sorted_roles = df['DevType'].copy().sum().sort_values(ascending=False).index.tolist()
sorted_roles

['Developer, full-stack',
 'Developer, back-end',
 '',
 'Developer, front-end',
 'Developer, desktop or enterprise applications',
 'Other (please specify):',
 'Developer, mobile',
 'Engineering manager',
 'Student',
 'Developer, embedded applications or devices',
 'Data scientist or machine learning specialist',
 'DevOps specialist',
 'Academic researcher',
 'Research & Development role',
 'Senior Executive (C-Suite, VP, etc.)',
 'Engineer, data',
 'Cloud infrastructure engineer',
 'Developer, game or graphics',
 'Data or business analyst',
 'System administrator',
 'Project manager',
 'Developer, QA or test',
 'Security professional',
 'Product manager',
 'Engineer, site reliability',
 'Educator',
 'Scientist',
 'Developer Experience',
 'Blockchain',
 'Hardware Engineer',
 'Designer',
 'Database administrator',
 'Developer Advocate',
 'Marketing or sales professional']

In [25]:
sorted_skills = df.drop('DevType', axis=1).sum().sort_values(ascending=False).droplevel(level=0).index.tolist()
sorted_skills

['Visual Studio Code',
 'JavaScript',
 'HTML/CSS',
 'Python',
 'SQL',
 'Docker',
 'npm',
 'PostgreSQL',
 'TypeScript',
 'Amazon Web Services (AWS)',
 'MySQL',
 'Node.js',
 'React',
 'Bash/Shell (all shells)',
 'Java',
 'Visual Studio',
 'C#',
 'SQLite',
 'Pip',
 'IntelliJ IDEA',
 'Notepad++',
 'C++',
 'MongoDB',
 'Microsoft SQL Server',
 'Vim',
 'Microsoft Azure',
 'Homebrew',
 'Yarn',
 '.NET (5+) ',
 'C',
 'Webpack',
 'Google Cloud',
 'PHP',
 'Make',
 'jQuery',
 'Redis',
 'Kubernetes',
 'Android Studio',
 'Express',
 'NumPy',
 'MariaDB',
 'Pandas',
 'PyCharm',
 'Angular',
 'NuGet',
 'Maven (build tool)',
 'Next.js',
 'Gradle',
 'PowerShell',
 'ASP.NET CORE',
 'Vite',
 'Vue.js',
 'Visual Studio Solution',
 'Go',
 'CMake',
 '.NET Framework (1.0 - 4.8)',
 'Rust',
 'Jupyter Notebook/JupyterLab',
 'Sublime Text',
 'Firebase',
 'Cloudflare',
 'Cargo',
 'Neovim',
 'Elasticsearch',
 'GNU GCC',
 'WordPress',
 'Digital Ocean',
 'ASP.NET',
 'Terraform',
 'Flask',
 'Spring Boot',
 'Eclipse',
 'MS

In [26]:
skills = []

for role in sorted_roles:

    role_mask =(df[('DevType',role)] == 1)

    skills_role = pd.concat({tech_col : df.loc[role_mask,tech_col].mean()*100 for tech_col in TECH_COLS})
    skills.append(skills_role)

skills = pd.concat(skills, axis=1)
skills.columns = sorted_roles
skills = skills.reset_index(level=0, drop=True)
skills = skills.loc[sorted_skills]
skills = skills.T

In [27]:
skills

,Visual Studio Code,JavaScript,HTML/CSS,Python,SQL,Docker,npm,PostgreSQL,TypeScript,Amazon Web Services (AWS),...,build2,TiDB,Raku,condo,tunit,snitch,lest,CUTE,ELFspy,liblittletest
"Developer, full-stack",79.331650,85.432291,71.839907,39.572567,62.393627,55.255489,64.581310,48.571984,59.848455,45.366233,...,0.097144,0.077715,0.077715,0.073829,0.038858,0.019429,0.003886,0.019429,0.023315,0.003886
"Developer, back-end",66.489633,50.105493,34.070571,44.285195,53.292106,63.150236,37.235358,53.866861,30.178247,51.858858,...,0.072754,0.203710,0.138232,0.058203,0.021826,0.021826,0.029101,0.014551,0.014551,0.000000
,66.626056,54.572775,52.631579,59.039961,31.018519,22.969461,34.665367,20.354126,25.487329,13.425926,...,0.251787,0.186810,0.227420,0.292398,0.129955,0.081222,0.089344,0.089344,0.121832,0.081222
"Developer, front-end",84.855058,91.619010,79.432065,21.159535,24.511931,36.935516,70.991915,27.824887,70.420035,33.484520,...,0.059160,0.039440,0.059160,0.019720,0.019720,0.039440,0.059160,0.039440,0.000000,0.000000
"Developer, desktop or enterprise applications",60.681352,41.393443,34.631148,37.115779,48.975410,27.330943,21.260246,23.565574,18.135246,21.209016,...,0.230533,0.128074,0.128074,0.128074,0.153689,0.128074,0.025615,0.076844,0.076844,0.051230
Other (please specify):,66.883117,54.967532,46.558442,53.246753,46.850649,38.928571,33.636364,33.441558,26.331169,33.506494,...,0.292208,0.292208,0.324675,0.194805,0.064935,0.064935,0.097403,0.064935,0.097403,0.129870
"Developer, mobile",61.301502,35.964575,24.874856,24.951868,23.488641,21.062765,26.068541,19.060454,19.098960,26.684636,...,0.115518,0.192530,0.154024,0.115518,0.154024,0.115518,0.115518,0.231036,0.115518,0.115518
Engineering manager,72.159370,65.863256,46.581407,47.024102,53.664535,62.469257,50.172159,52.090507,44.663060,60.452533,...,0.147565,0.098377,0.049188,0.098377,0.049188,0.000000,0.049188,0.000000,0.000000,0.000000
Student,79.358717,67.334669,63.827655,70.490982,43.036072,33.316633,45.641283,29.959920,34.168337,17.935872,...,0.350701,0.250501,0.551102,0.450902,0.150301,0.150301,0.200401,0.250501,0.300601,0.200401
"Developer, embedded applications or devices",68.997290,30.027100,22.493225,68.726287,19.403794,42.493225,15.447154,18.970190,12.520325,19.295393,...,0.487805,0.000000,0.108401,0.000000,0.000000,0.054201,0.108401,0.000000,0.000000,0.000000


In [28]:
fig = go.Figure(data=go.Heatmap(z=skills, x=skills.columns,y=skills.index, colorscale='magma', ygap=10))
fig.update_layout(width=2500, height=1000)
fig.show()
fig.write_html(f'{IMAGES_PATH}/01_skills_heatmap.html')

In [29]:
from sklearn.preprocessing import StandardScaler
std_skills = StandardScaler().fit_transform(skills)
std_skills = pd.DataFrame(std_skills, columns=skills.columns, index=skills.index)
fig = go.Figure(data=go.Heatmap(z=std_skills, x=skills.columns,y=skills.index, colorscale='magma', ygap=1))
fig.update_layout(width=2500, height=700)
fig.show()
fig.write_html(f'{IMAGES_PATH}/02_normalized_skills_heatmap.html')

In [30]:
sorted_roles

['Developer, full-stack',
 'Developer, back-end',
 '',
 'Developer, front-end',
 'Developer, desktop or enterprise applications',
 'Other (please specify):',
 'Developer, mobile',
 'Engineering manager',
 'Student',
 'Developer, embedded applications or devices',
 'Data scientist or machine learning specialist',
 'DevOps specialist',
 'Academic researcher',
 'Research & Development role',
 'Senior Executive (C-Suite, VP, etc.)',
 'Engineer, data',
 'Cloud infrastructure engineer',
 'Developer, game or graphics',
 'Data or business analyst',
 'System administrator',
 'Project manager',
 'Developer, QA or test',
 'Security professional',
 'Product manager',
 'Engineer, site reliability',
 'Educator',
 'Scientist',
 'Developer Experience',
 'Blockchain',
 'Hardware Engineer',
 'Designer',
 'Database administrator',
 'Developer Advocate',
 'Marketing or sales professional']

In [31]:
skills.loc['Engineer, data']

Visual Studio Code    69.230769
JavaScript            35.817308
HTML/CSS              28.445513
Python                85.416667
SQL                   72.355769
                        ...    
snitch                 0.160256
lest                   0.080128
CUTE                   0.080128
ELFspy                 0.000000
liblittletest          0.000000
Name: Engineer, data, Length: 270, dtype: float64

In [32]:
#role = np.random.choice(sorted_roles)
# role = 'Developer, embedded applications or devices'
# role = 'Data scientist or machine learning specialist'
# role = 'Developer, back-end'
role = 'Developer, mobile'
single_role_skills = pd.concat([skills.loc[role], std_skills.loc[role]], axis=1)
single_role_skills.columns = ['percentage', 'specificity']
single_role_skills = single_role_skills.sort_values('percentage')
#single_role_skills
threshold = 25

single_role_skills = single_role_skills[single_role_skills['percentage'] > threshold]

fig = px.bar(df,
             y=single_role_skills.index,
             x=single_role_skills['percentage'],
             color=single_role_skills['specificity'],
             color_continuous_scale='orrd',
             range_color=[std_skills.values.min(),std_skills.values.max()],
             orientation='h')

fig.update_layout(width=800, height=800, title=role)
fig.show()